In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
import os
import codecs
import pandas as pd
import html2text
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from textblob import TextBlob
from nltk.corpus import stopwords
from wordcloud import WordCloud
import seaborn
from nltk import word_tokenize, pos_tag
import ast
from operator import itemgetter
from gensim.models import LdaModel


In [2]:
jokes_dataframe = pd.DataFrame()

In [3]:
def load_clean_joke():

    ret_jokes = []

    for i in range(1,101):
        file = 'init'+str(i)+'.html'
        data = codecs.open('jokes/' + file, 'r', encoding="cp1252")
        joke_html = data.read()

        # Extracting joke
        joke = html2text.html2text(joke_html)
        # Extracting joke_id
        joke_id = int(file.split('init')[1].split('.html')[0])
        cleaned_string = re.sub(r'[\|]+|[-]+', '', joke)

        ret_jokes.append(cleaned_string.strip())
    
    return ret_jokes


In [4]:
Jokes = load_clean_joke()

In [5]:
jokes_dataframe["Original Jokes"] = Jokes

In [6]:
from nltk.corpus import stopwords

def remove_tags_puntuatuions_tags(Joke):
    special_char_patterns = r'[^a-zA-Z0-9\s]'
    urlPatterns = r'http\S+|www\S+'

    text = Joke

    text = re.sub(special_char_patterns, ' ', text)
    text = re.sub(urlPatterns, ' ', text)
    text = re.sub(r'\n', ' ', text)

    return text

def remove_stop_words_Tokenization(Joke):
    
    stopwords_ = set(stopwords.words('english'))
    Tokenize_Joke = []
    for word in Joke.split():
        if word not in stopwords_:
            Tokenize_Joke.append(word)

    return Tokenize_Joke

def lower(Joke):
    ret_lst = [word.lower() for word in Joke]
    return ret_lst

def preprocess_clean_jokes(uncleaned_Jokes):

    Cleaned_Jokes = []

    for Joke in uncleaned_Jokes:
        # Remove puntuations and tags from the data
        Joke = remove_tags_puntuatuions_tags(Joke)
        # Correct Spelling
        Joke = TextBlob(Joke)
        # Remove Stop Words and do Tokenization of Joke
        Joke = remove_stop_words_Tokenization(Joke)

        Cleaned_Jokes.append(lower(Joke))

    return Cleaned_Jokes



In [7]:
preprocessed_jokes = preprocess_clean_jokes(Jokes)

In [8]:
jokes_dataframe["PreProcessed Jokes"] = preprocessed_jokes

In [9]:
from nltk.stem import PorterStemmer

def stem_tokens(tokenized_text):
    stemmer = PorterStemmer()
    stemmed_tokens = []
    for preprocessed_joke in tokenized_text:
        stemmed_tokens.append([stemmer.stem(token) for token in preprocessed_joke])
    
    return stemmed_tokens

In [10]:
import nltk
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
        
    lemmatized_text = " ".join([lemmatizer.lemmatize(word) for word in text])
    
    return lemmatized_text

In [11]:
lemmitized_jokes = [lemmatize_text(text) for text in preprocessed_jokes]

In [12]:
jokes_dataframe["lemmitized_jokes"] = lemmitized_jokes 

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

def feature_Extraction_TF_IDF(corpus):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    feature_names = tfidf_vectorizer.get_feature_names_out()
    tfidf_array = tfidf_matrix.toarray()

    return tfidf_array


In [14]:
TF_IDF_lemma = feature_Extraction_TF_IDF(lemmitized_jokes)

In [15]:
jokes_dataframe["TF_IDF_Lemma"] = [i for i in TF_IDF_lemma] 

In [27]:
cleaned_list = list(jokes_dataframe["PreProcessed Jokes"])

merged_lst = []

for joke in cleaned_list:
    merged_lst.append(" ".join(joke))



In [28]:
jokes_dataframe["merged cleaned"] = merged_lst

In [29]:
jokes_dataframe

,Original Jokes,PreProcessed Jokes,lemmitized_jokes,TF_IDF_Lemma,merged cleaned
0,"A man visits the doctor. The doctor says ""I ha...","[a, man, visits, doctor, the, doctor, says, i,...",a man visit doctor the doctor say i bad news y...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a man visits doctor the doctor says i bad news...
1,This couple had an excellent relationship goin...,"[this, couple, excellent, relationship, going,...",this couple excellent relationship going one d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",this couple excellent relationship going one d...
2,Q. What's 200 feet long and has 4 teeth?\n\nA....,"[q, what, 200, feet, long, 4, teeth, a, the, f...",q what 200 foot long 4 teeth a the front row w...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what 200 feet long 4 teeth a the front row w...
3,Q. What's the difference between a man and a t...,"[q, what, difference, man, toilet, a, a, toile...",q what difference man toilet a a toilet follow...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what difference man toilet a a toilet follow...
4,Q. What's O. J. Simpson's Internet address?\n\...,"[q, what, o, j, simpson, internet, address, a,...",q what o j simpson internet address a slash sl...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what o j simpson internet address a slash sl...
...,...,...,...,...,...
95,Two attorneys went into a diner and ordered tw...,"[two, attorneys, went, diner, ordered, two, dr...",two attorney went diner ordered two drink then...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",two attorneys went diner ordered two drinks th...
96,A teacher is explaining to her class how diffe...,"[a, teacher, explaining, class, different, lan...",a teacher explaining class different language ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a teacher explaining class different languages...
97,Age and Womanhood\n\n1\. Between the ages of 1...,"[age, womanhood, 1, between, ages, 13, 18, she...",age womanhood 1 between age 13 18 she like afr...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10500478...",age womanhood 1 between ages 13 18 she like af...
98,A bus station is where a bus stops. \nA train...,"[a, bus, station, bus, stops, a, train, statio...",a bus station bus stop a train station train s...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a bus station bus stops a train station train ...


In [37]:
import nltk

def posTagging(text):
    return nltk.pos_tag(text, tagset='universal')

jokes_dataframe['pos_tags'] = jokes_dataframe['PreProcessed Jokes'].apply(lambda x: posTagging(x))
jokes_dataframe.head(2)

,Original Jokes,PreProcessed Jokes,lemmitized_jokes,TF_IDF_Lemma,merged cleaned,pos_tags
0,"A man visits the doctor. The doctor says ""I ha...","[a, man, visits, doctor, the, doctor, says, i,...",a man visit doctor the doctor say i bad news y...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a man visits doctor the doctor says i bad news...,"[(a, DET), (man, NOUN), (visits, VERB), (docto..."
1,This couple had an excellent relationship goin...,"[this, couple, excellent, relationship, going,...",this couple excellent relationship going one d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",this couple excellent relationship going one d...,"[(this, DET), (couple, ADJ), (excellent, NOUN)..."


In [38]:
pd.options.mode.chained_assignment = None

In [41]:
jokes_dataframe["pos_tags"][0]

[('a', 'DET'),
 ('man', 'NOUN'),
 ('visits', 'VERB'),
 ('doctor', 'VERB'),
 ('the', 'DET'),
 ('doctor', 'NOUN'),
 ('says', 'VERB'),
 ('i', 'NOUN'),
 ('bad', 'ADJ'),
 ('news', 'NOUN'),
 ('you', 'PRON'),
 ('cancer', 'NOUN'),
 ('alzheimer', 'VERB'),
 ('disease', 'ADP'),
 ('the', 'DET'),
 ('man', 'NOUN'),
 ('replies', 'VERB'),
 ('well', 'ADV'),
 ('thank', 'ADJ'),
 ('god', 'NOUN'),
 ('i', 'NOUN'),
 ('cancer', 'NOUN')]

In [42]:
jokes_dataframe['ADJ'] = pd.Series(dtype=str)
jokes_dataframe['ADP'] = pd.Series(dtype=str)
jokes_dataframe['ADV'] = pd.Series(dtype=str)
jokes_dataframe['CONJ'] = pd.Series(dtype=str)
jokes_dataframe['DET'] = pd.Series(dtype=str)
jokes_dataframe['NOUN'] = pd.Series(dtype=str)
jokes_dataframe['NUM'] = pd.Series(dtype=str)
jokes_dataframe['PRT'] = pd.Series(dtype=str)
jokes_dataframe['PRON'] = pd.Series(dtype=str)
jokes_dataframe['PRT'] = pd.Series(dtype=str)
jokes_dataframe['PRON'] = pd.Series(dtype=str)
jokes_dataframe['VERB'] = pd.Series(dtype=str)
jokes_dataframe['PUNC'] = pd.Series(dtype=str)
jokes_dataframe['OTHERS'] = pd.Series(dtype=str)

In [56]:
def aggregate_tags(col_tags, tag_columns = {
    'ADJ': 'ADJ',
    'ADP': 'ADP',
    'ADV': 'ADV',
    'CONJ': 'CONJ',
    'DET': 'DET',
    'NOUN': 'NOUN',
    'NUM': 'NUM',
    'PRT': 'PRT',
    'PRON': 'PRON',
    'VERB': 'VERB',
    '.': '.',
    'X': 'X'}):
    aggregated_tags = {col: [] for col in tag_columns.values()}
    
    ret_frame = pd.DataFrame(columns=aggregated_tags)

    for i,joke_tags in enumerate(col_tags):
        temp_dict = {val:[] for val in aggregated_tags.keys()}
        for tag in joke_tags:
            temp_dict[tag[1]].append(tag[0])

        ret_frame = pd.concat([ret_frame, pd.DataFrame([temp_dict])], ignore_index=True)

    return ret_frame

aggregate_tags_frame = aggregate_tags(jokes_dataframe["pos_tags"])


In [57]:
aggregate_tags_frame

,ADJ,ADP,ADV,CONJ,DET,NOUN,NUM,PRT,PRON,VERB,.,X
0,"[bad, thank]",[disease],[well],[],"[a, the, the]","[man, doctor, i, news, cancer, man, god, i, ca...",[],[],[you],"[visits, doctor, says, alzheimer, replies]",[],[]
1,"[couple, girlfriend, awful, big, old]",[that],"[told, possibly, awfully]",[],[this],"[excellent, relationship, day, home, work, pac...","[one, ten]",[],"[he, what, they, he]","[going, came, find, asked, leaving, heard, cou...",[],[]
2,"[long, nelson]","[teeth, willie]",[],[],"[a, the]","[feet, front, row, concert]","[200, 4]",[],[what],[q],[],[]
3,[],[around],[],[],"[a, a]","[difference, man, toilet, follow, use]",[],[],[what],"[q, toilet]",[],[]
4,[slash],[o],[],[],[a],"[j, simpson, internet, address, slash, backsla...",[],[],[what],[q],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...
95,[shrugged],"[eat, told]","[diner, then, quite]",[],"[the, the]","[attorneys, drinks, sandwiches, briefcases, ow...","[two, two]",[],[you],"[went, ordered, produced, started, became, con...",[],[]
96,"[different, positive, negative, negative, posi...","[in, in]","[differently, together]",[but],[a],"[teacher, class, languages, negatives, languag...","[two, two, one]",[],[she],"[explaining, use, says, followed, followed, ma...",[],[]
97,"[hot, exotic, beautiful, free]","[between, like, between, like, between, like, ...","[fully, breathtakingly, still]",[],[],"[age, ages, africa, virgin, ages, asia, ages, ...","[1, 13, 18, 2, 19, 35, 3, 36, 45, 4, 46, 56, 5...",[],"[she, she, she, she]","[womanhood, unexplored, explored, exhausted, p...",[],[]
98,[],[on],[],[],"[a, a]","[bus, station, bus, train, station, train, sto...",[],[],[],[stops],[],[]


pos tagging: https://212digital.medium.com/an-introduction-to-part-of-speech-tagging-what-it-is-and-how-you-can-use-it-in-natural-language-9723f4696f78

sentiment analysis — By identifying words with positive or negative connotations, POS tagging can be used to calculate the overall sentiment of a piece of text.

topic identification — By looking at which words are most commonly used together, POS tagging can help automatically identify the main topics of a document.

In [124]:
def filterByPOS(tags):
    print("Before eval:", tags)
    tags = tags[1:len(tags)-1]
    print("After slicing:", tags)
    
    try:
        tags = ast.literal_eval(tags)
    except ValueError as e:
        print("Error during literal_eval:", e)
    
    print("After eval:", tags)
    
    txt = []
    
    for t in tags:
        if t[1] in ['NOUN', 'ADV'] and len(t[0]) > 1:
            txt.append(t[0])
            
    return ' '.join(txt)

helper = jokes_dataframe['pos_tags'].apply(lambda x: filterByPOS(x))


Before eval: [('a', 'DET'), ('man', 'NOUN'), ('visits', 'VERB'), ('doctor', 'VERB'), ('the', 'DET'), ('doctor', 'NOUN'), ('says', 'VERB'), ('i', 'NOUN'), ('bad', 'ADJ'), ('news', 'NOUN'), ('you', 'PRON'), ('cancer', 'NOUN'), ('alzheimer', 'VERB'), ('disease', 'ADP'), ('the', 'DET'), ('man', 'NOUN'), ('replies', 'VERB'), ('well', 'ADV'), ('thank', 'ADJ'), ('god', 'NOUN'), ('i', 'NOUN'), ('cancer', 'NOUN')]
After slicing: [('man', 'NOUN'), ('visits', 'VERB'), ('doctor', 'VERB'), ('the', 'DET'), ('doctor', 'NOUN'), ('says', 'VERB'), ('i', 'NOUN'), ('bad', 'ADJ'), ('news', 'NOUN'), ('you', 'PRON'), ('cancer', 'NOUN'), ('alzheimer', 'VERB'), ('disease', 'ADP'), ('the', 'DET'), ('man', 'NOUN'), ('replies', 'VERB'), ('well', 'ADV'), ('thank', 'ADJ'), ('god', 'NOUN'), ('i', 'NOUN')]
Error during literal_eval: malformed node or string: [('man', 'NOUN'), ('visits', 'VERB'), ('doctor', 'VERB'), ('the', 'DET'), ('doctor', 'NOUN'), ('says', 'VERB'), ('i', 'NOUN'), ('bad', 'ADJ'), ('news', 'NOUN'), 

Applying topic modelling


In [125]:
from gensim.corpora import Dictionary

docs = [d.split() for d in helper]

dictionary = Dictionary(docs)

corpus = [dictionary.doc2bow(doc) for doc in docs]

In [126]:
print(dictionary)

Dictionary(799 unique tokens: ['cancer', 'doctor', 'god', 'man', 'news']...)


In [127]:
temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=dictionary.id2token,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=400,
    num_topics=10,
    passes=20,
    eval_every=None
)

top_topics = model.top_topics(corpus)
top_topics

[([(0.03799817, 'ages'),
   (0.019462394, 'chukcha'),
   (0.01946239, 'guard'),
   (0.01019448, 'resources'),
   (0.01019448, 'everybody'),
   (0.01019448, 'still'),
   (0.01019448, 'virgin'),
   (0.01019448, 'europe'),
   (0.01019448, 'breathtakingly'),
   (0.01019448, 'asia'),
   (0.01019448, 'africa'),
   (0.0101944795, 'america'),
   (0.0101944795, 'interest'),
   (0.0101944795, 'fully'),
   (0.0101944795, 'australia'),
   (0.010194476, 'phones'),
   (0.010194476, 'eskimo'),
   (0.010194476, 'idiot'),
   (0.010194476, 'member'),
   (0.010194476, 'parliament')],
  -11.086540997387582),
 ([(0.0858451, 'blah'),
   (0.028993946, 'course'),
   (0.023308832, 'degrees'),
   (0.023308832, 'ship'),
   (0.023308827, 'engineers'),
   (0.011938589, 'right'),
   (0.011938589, 'clothes'),
   (0.011938589, 'floor'),
   (0.011938589, 'collision'),
   (0.011938589, 'north'),
   (0.011938585, 'swartzeneger'),
   (0.0062534413, 'authorities'),
   (0.0062534413, 'captain'),
   (0.0062534413, 'canadian

This above code extracts the top topics from the trained LDA model using the top_topics method. Each topic is 
represented as a list of tuples, where each tuple contains a word and its associated weight in the topic.

In [128]:
cluster = []

for d in helper:
    bow = dictionary.doc2bow(d.split())
    topics = model.get_document_topics(bow, minimum_probability = 0.0)
    cluster.append(topics)
    
jokes_dataframe['cluster'] = pd.Series(cluster)

In [129]:
print(jokes_dataframe["cluster"][0])

[(0, 0.005692455), (1, 0.0067563336), (2, 0.0040689902), (3, 0.0044702217), (4, 0.005104626), (5, 0.0056291563), (6, 0.0046943435), (7, 0.0055746767), (8, 0.9543301), (9, 0.0036790909)]


In [130]:
sorted_topics = []

for i in range(len(jokes_dataframe)):
    sorted_topics.append(sorted(jokes_dataframe['cluster'][i],key=itemgetter(1),  reverse=True))
    
jokes_dataframe['sorted_topics'] = pd.Series(sorted_topics)
jokes_dataframe.head()

# jokes_dataframe.to_csv("data/jokes_sorted_topic.csv", index=False)

,Original Jokes,PreProcessed Jokes,lemmitized_jokes,TF_IDF_Lemma,merged cleaned,pos_tags,cluster,sorted_topics,main_topic
0,"A man visits the doctor. The doctor says ""I ha...","[a, man, visits, doctor, the, doctor, says, i,...",a man visit doctor the doctor say i bad news y...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a man visits doctor the doctor says i bad news...,"[(a, DET), (man, NOUN), (visits, VERB), (docto...","[(0, 0.005692455), (1, 0.0067563336), (2, 0.00...","[(8, 0.9543301), (1, 0.0067563336), (0, 0.0056...",0
1,This couple had an excellent relationship goin...,"[this, couple, excellent, relationship, going,...",this couple excellent relationship going one d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",this couple excellent relationship going one d...,"[(this, DET), (couple, ADJ), (excellent, NOUN)...","[(0, 0.002922212), (1, 0.0034683526), (2, 0.00...","[(3, 0.9756108), (1, 0.0034683526), (8, 0.0032...",3
2,Q. What's 200 feet long and has 4 teeth?\n\nA....,"[q, what, 200, feet, long, 4, teeth, a, the, f...",q what 200 foot long 4 teeth a the front row w...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what 200 feet long 4 teeth a the front row w...,"[(q, VERB), (what, PRON), (200, NUM), (feet, N...","[(0, 0.012421779), (1, 0.014743321), (2, 0.008...","[(7, 0.89873606), (1, 0.014743321), (8, 0.0137...",4
3,Q. What's the difference between a man and a t...,"[q, what, difference, man, toilet, a, a, toile...",q what difference man toilet a a toilet follow...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what difference man toilet a a toilet follow...,"[(q, VERB), (what, PRON), (difference, NOUN), ...","[(0, 0.009588222), (1, 0.9238259), (2, 0.00685...","[(1, 0.9238259), (8, 0.010629054), (0, 0.00958...",9
4,Q. What's O. J. Simpson's Internet address?\n\...,"[q, what, o, j, simpson, internet, address, a,...",q what o j simpson internet address a slash sl...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what o j simpson internet address a slash sl...,"[(q, VERB), (what, PRON), (o, ADP), (j, NOUN),...","[(0, 0.0056924727), (1, 0.006756355), (2, 0.00...","[(4, 0.95312417), (1, 0.006756355), (8, 0.0063...",6


In [131]:
maximum_probability_topic = []
for i in range(len(jokes_dataframe)):
    maximum_probability_topic.append(jokes_dataframe['sorted_topics'][i][0][0])

jokes_dataframe['main_topic'] = maximum_probability_topic
# jokes1.to_csv("data/jokes_sorted_topic.csv", index=False)

In [132]:
jokes_dataframe

,Original Jokes,PreProcessed Jokes,lemmitized_jokes,TF_IDF_Lemma,merged cleaned,pos_tags,cluster,sorted_topics,main_topic
0,"A man visits the doctor. The doctor says ""I ha...","[a, man, visits, doctor, the, doctor, says, i,...",a man visit doctor the doctor say i bad news y...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a man visits doctor the doctor says i bad news...,"[(a, DET), (man, NOUN), (visits, VERB), (docto...","[(0, 0.005692455), (1, 0.0067563336), (2, 0.00...","[(8, 0.9543301), (1, 0.0067563336), (0, 0.0056...",8
1,This couple had an excellent relationship goin...,"[this, couple, excellent, relationship, going,...",this couple excellent relationship going one d...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",this couple excellent relationship going one d...,"[(this, DET), (couple, ADJ), (excellent, NOUN)...","[(0, 0.002922212), (1, 0.0034683526), (2, 0.00...","[(3, 0.9756108), (1, 0.0034683526), (8, 0.0032...",3
2,Q. What's 200 feet long and has 4 teeth?\n\nA....,"[q, what, 200, feet, long, 4, teeth, a, the, f...",q what 200 foot long 4 teeth a the front row w...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what 200 feet long 4 teeth a the front row w...,"[(q, VERB), (what, PRON), (200, NUM), (feet, N...","[(0, 0.012421779), (1, 0.014743321), (2, 0.008...","[(7, 0.89873606), (1, 0.014743321), (8, 0.0137...",7
3,Q. What's the difference between a man and a t...,"[q, what, difference, man, toilet, a, a, toile...",q what difference man toilet a a toilet follow...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what difference man toilet a a toilet follow...,"[(q, VERB), (what, PRON), (difference, NOUN), ...","[(0, 0.009588222), (1, 0.9238259), (2, 0.00685...","[(1, 0.9238259), (8, 0.010629054), (0, 0.00958...",1
4,Q. What's O. J. Simpson's Internet address?\n\...,"[q, what, o, j, simpson, internet, address, a,...",q what o j simpson internet address a slash sl...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",q what o j simpson internet address a slash sl...,"[(q, VERB), (what, PRON), (o, ADP), (j, NOUN),...","[(0, 0.0056924727), (1, 0.006756355), (2, 0.00...","[(4, 0.95312417), (1, 0.006756355), (8, 0.0063...",4
...,...,...,...,...,...,...,...,...,...
95,Two attorneys went into a diner and ordered tw...,"[two, attorneys, went, diner, ordered, two, dr...",two attorney went diner ordered two drink then...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",two attorneys went diner ordered two drinks th...,"[(two, NUM), (attorneys, NOUN), (went, VERB), ...","[(0, 0.0036923173), (1, 0.004382385), (2, 0.00...","[(8, 0.9703769), (1, 0.004382385), (0, 0.00369...",8
96,A teacher is explaining to her class how diffe...,"[a, teacher, explaining, class, different, lan...",a teacher explaining class different language ...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a teacher explaining class different languages...,"[(a, DET), (teacher, NOUN), (explaining, VERB)...","[(0, 0.9777825), (1, 0.0032429425), (2, 0.0019...","[(0, 0.9777825), (1, 0.0032429425), (8, 0.0030...",0
97,Age and Womanhood\n\n1\. Between the ages of 1...,"[age, womanhood, 1, between, ages, 13, 18, she...",age womanhood 1 between age 13 18 she like afr...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10500478...",age womanhood 1 between ages 13 18 she like af...,"[(age, NOUN), (womanhood, VERB), (1, NUM), (be...","[(0, 0.0025654505), (1, 0.003044915), (2, 0.00...","[(9, 0.97823185), (1, 0.003044915), (8, 0.0028...",9
98,A bus station is where a bus stops. \nA train...,"[a, bus, station, bus, stops, a, train, statio...",a bus station bus stop a train station train s...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",a bus station bus stops a train station train ...,"[(a, DET), (bus, NOUN), (station, NOUN), (bus,...","[(0, 0.004479278), (1, 0.9644142), (2, 0.00320...","[(1, 0.9644142), (8, 0.0049655177), (0, 0.0044...",1


In [134]:
top_topics[8]

([(0.033070195, 'man'),
  (0.027604032, 'woman'),
  (0.02213786, 'engineer'),
  (0.016671687, 'back'),
  (0.016671687, 'car'),
  (0.011205514, 'water'),
  (0.011205513, 'look'),
  (0.011205511, 'wow'),
  (0.011205507, 'how'),
  (0.008472424, 'lawyer'),
  (0.008472424, 'wakes'),
  (0.008472424, 'room'),
  (0.008472423, 'teller'),
  (0.008472423, 'guy'),
  (0.008472417, 'light'),
  (0.00573933, 'pope'),
  (0.005739329, 'arm'),
  (0.005739329, 'bmw'),
  (0.005739329, 'problem'),
  (0.005739329, 'officer')],
 -17.914956677900072)